<a href="https://colab.research.google.com/github/6rownGuy/ai_poet/blob/main/ai_poet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping

import numpy as np 

In [2]:
df = pd.read_csv('poem_data.csv')
df.head()

,author,content,poem name,age,type
0,WILLIAM SHAKESPEARE,Let the bird of loudest lay\r\nOn the sole Ara...,The Phoenix and the Turtle,Renaissance,Mythology & Folklore
1,DUCHESS OF NEWCASTLE MARGARET CAVENDISH,"Sir Charles into my chamber coming in,\r\nWhen...",An Epilogue to the Above,Renaissance,Mythology & Folklore
2,THOMAS BASTARD,"Our vice runs beyond all that old men saw,\r\n...","Book 7, Epigram 42",Renaissance,Mythology & Folklore
3,EDMUND SPENSER,"Lo I the man, whose Muse whilome did maske,\r\...","from The Faerie Queene: Book I, Canto I",Renaissance,Mythology & Folklore
4,RICHARD BARNFIELD,"Long have I longd to see my love againe,\r\nSt...",Sonnet 16,Renaissance,Mythology & Folklore


In [3]:
all_poems = list(df['content'])
all_poems = '\n'.join(all_poems[:51])

In [4]:
import re

dirty_corpus = re.split("; |, |\n|\r\n", all_poems.lower())

corpus = []
for line in dirty_corpus:
    if line == '':
        continue
    corpus.append(line)

In [5]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

print(tokenizer.word_index)
print(total_words)

{'the': 1, 'and': 2, 'to': 3, 'of': 4, 'that': 5, 'in': 6, 'her': 7, 'a': 8, 'with': 9, 'his': 10, 'i': 11, 'all': 12, 'for': 13, 'he': 14, 'as': 15, 'my': 16, 'so': 17, 'is': 18, 'which': 19, 'but': 20, 'be': 21, 'not': 22, 'thy': 23, 'did': 24, 'thou': 25, 'they': 26, 'was': 27, 'their': 28, 'your': 29, 'it': 30, 'love': 31, 'she': 32, 'when': 33, 'or': 34, 'from': 35, 'ye': 36, 'me': 37, 'him': 38, 'this': 39, 'doth': 40, 'them': 41, 'now': 42, 'then': 43, 'you': 44, 'by': 45, 'let': 46, 'no': 47, 'day': 48, 'may': 49, 'on': 50, 'like': 51, 'more': 52, 'long': 53, 'there': 54, 'one': 55, 'such': 56, 'where': 57, 'had': 58, 'do': 59, 'have': 60, 'at': 61, 'yet': 62, 'were': 63, 'how': 64, 'upon': 65, 'will': 66, 'whose': 67, 'sing': 68, 'nor': 69, 'thee': 70, 'can': 71, 'well': 72, 'see': 73, 'out': 74, 'night': 75, 'unto': 76, 'our': 77, 'fair': 78, 'still': 79, 'doe': 80, 'if': 81, 'sweet': 82, 'great': 83, 'go': 84, 'what': 85, 'we': 86, 'some': 87, 'hath': 88, 'shall': 89, 'ever'

In [6]:
# input sequences will store the n-grams

input_sequences = []

for line in corpus:
    
    # tokenize each sentence
    token_list = tokenizer.texts_to_sequences([line])[0]
    
    # generate n-grams of the tokens and store them in
    # input_sequences
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [7]:
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [8]:
# create and compile model

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

earlystop = EarlyStopping(monitor='loss', min_delta=0, patience=5, verbose=0, mode='min')



In [9]:
history = model.fit(
    x=xs,
    y=ys,
    epochs=100,
    verbose=1,
    callbacks=[earlystop]
)

Epoch 1/100
513/513 [==============================] - 5s 10ms/step - loss: 7.4924 - accuracy: 0.0355
Epoch 2/100
513/513 [==============================] - 5s 10ms/step - loss: 6.6681 - accuracy: 0.0607
Epoch 3/100
513/513 [==============================] - 5s 10ms/step - loss: 5.7749 - accuracy: 0.0955
Epoch 4/100
513/513 [==============================] - 5s 10ms/step - loss: 4.6405 - accuracy: 0.1667
Epoch 5/100
513/513 [==============================] - 5s 10ms/step - loss: 3.5882 - accuracy: 0.2924
Epoch 6/100
513/513 [==============================] - 5s 10ms/step - loss: 2.8257 - accuracy: 0.4118
Epoch 7/100
513/513 [==============================] - 5s 10ms/step - loss: 2.2910 - accuracy: 0.5096
Epoch 8/100
513/513 [==============================] - 5s 10ms/step - loss: 1.9613 - accuracy: 0.5704
Epoch 9/100
513/513 [==============================] - 5s 10ms/step - loss: 1.7095 - accuracy: 0.6262
Epoch 10/100
513/513 [==============================] - 5s 10ms/step - loss: 1.576

In [10]:
#print model.summary()
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 19, 100)           500000    
_________________________________________________________________
lstm (LSTM)                  (None, 150)               150600    
_________________________________________________________________
dense (Dense)                (None, 5000)              755000    
Total params: 1,405,600
Trainable params: 1,405,600
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
seed_text = "Let the bird of"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

Let the bird of loudest lay to sing carefull repent there but read as it is thy delight to chace the bridal day shall see a frenne spright fell place defiled has the knight at carowe and colde bound called rosalinde with muse doe ye sleepe thus long as fast as time sings those still cease our hopefull hap to sing ye faire front a poets pen may write her to constraine shore the thine to thirst with sweet furnace as are light are partes it is best contentment aloud did win to speake seek themselves were fain in wonder at of this were
